# VacationPy
----



In [48]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [49]:

cities_data_to_load = "output_data/cities.csv"
cities_weather_df = pd.read_csv(cities_data_to_load)
cities_weather_df = pd.DataFrame(cities_weather_df)
cities_weather_df['City']=cities_weather_df['City'].str.capitalize()
cities_weather_df['Country']=cities_weather_df['Country'].str.upper()
cities_weather_df

,City ID,City,Country,Act Lat,Act Lon,Temperature (F),Humidity,Cloudiness,Wind Speed
0,212730,Kisangani,CD,0.52,25.20,75.42,75,61,3.67
1,1501377,Kungurtug,RU,50.60,97.52,27.77,58,0,4.23
2,2163355,Hobart,AU,-42.88,147.33,40.64,87,0,9.17
3,3652764,Puerto ayora,EC,-0.74,-90.35,84.13,70,20,9.17
4,3137469,Sorland,NO,67.67,12.69,37.40,86,20,12.75
...,...,...,...,...,...,...,...,...,...
496,361058,Alexandria,EG,31.22,29.96,61.21,67,8,8.05
497,1218612,Murgab,TM,37.50,61.97,39.20,86,90,6.93
498,2038080,Chengzihe,CN,45.33,131.07,33.01,86,95,6.49
499,3356832,Henties bay,NA,-22.12,14.28,60.01,91,19,4.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

city_humidity_map_data = cities_weather_df[['Act Lat', 'Act Lon', 'Humidity']]
minneapolis = [44.9778, -93.265]
locations = city_humidity_map_data[['Act Lat', 'Act Lon']]
weights = city_humidity_map_data['Humidity']

figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(map_type='HYBRID', center = minneapolis,zoom_level=2, layout=figure_layout)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100, point_radius=1.5))
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
#Function to test for valid user inputs.
#question = the question you want to display to prompt input
#valid_lst = a list of valid responses
#allowed_tries = how many attempts to allow the user to give a valid response
#default_input = the input value to default to if the user fails to give a valid input in the tries allowed
#Converts numbers to strings for checking and returns the user_input as an upper case string.
#Use upper() on valid_list and user_input to avoid upper/lower case problems.
#Returns a boolean of whether the user provided a valid input (TRUE) and the final input value (which is the default_input if FALSE)

def valid_input(question, valid_lst, err_msg, allowed_tries, default_input):
    try_count = 0
    while True:
        user_input = input(question+" ").upper()
        if user_input in str(valid_lst).upper():
            Valid = True
            break
        else:
            try_count+=1
            print(f'Invalid input {allowed_tries - try_count} attempts remaining.')
            print(err_msg)
            if try_count >= allowed_tries:
                Valid = False
                user_input = str(default_input)
                break
    return Valid, user_input

In [105]:
custom_q = 'Default vacation spot search criteria are: 70F, 40% humidity, and any level of windiness or cloudiness. Would you like to customize your search (y/n)'
custom_allowed = ['y', 'n']
custom_err_msg = 'Please enter y or n.'
custom_tries = 3
custom_default = 'N'
custom = valid_input(custom_q, custom_allowed, custom_err_msg, custom_tries, custom_default)

if custom[1]=='Y':
    #Get user preference for vacation spot temp
    user_temp_q = 'What is your ideal temperature in F?'
    user_temp_allowed = np.arange(-40, 120)
    user_temp_err_msg = 'Please enter a number -40 to 120'
    user_temp_tries = 3
    user_temp_default = 70
    user_temp_input = valid_input(user_temp_q, user_temp_allowed, user_temp_err_msg, user_temp_tries, user_temp_default)
    ideal_temp = int(user_temp_input[1])

    #Get user preference for vacation spot humidity
    user_humid_q = 'What is your ideal humidity level? 30-50 is considered comfortable. Enter I if you are indifferent.'
    user_humid_allowed = [np.arange(0, 101), 'I']
    user_humid_err_msg = 'Please enter a number 0 to 100'
    user_humid_tries = 3
    user_humid_default = 40
    user_humid_input = valid_input(user_humid_q, user_humid_allowed, user_humid_err_msg, user_humid_tries, user_humid_default)
    ideal_humid = user_humid_input[1]

    #Get user preference for vacation spot wind
    user_wind_q = 'How much wind is ok? (G)entle wind, (F)resh wind, (I)ndifferent'
    user_wind_allowed = ['G', 'F', 'I']
    user_wind_err_msg = 'Please enter G, F, or I'
    user_wind_tries = 3
    user_wind_default = 'I'
    user_wind_input = valid_input(user_wind_q, user_wind_allowed, user_wind_err_msg, user_wind_tries, user_wind_default)
    ideal_wind = user_wind_input[1]

    #Get user preference for vacation spot cloudiness
    user_clouds_q = 'How much cloudiness is ok? (C)lear, (S)cattered, or (I)ndifferent'
    user_clouds_allowed = ['C', 'S', 'I']
    user_clouds_err_msg = 'Please enter C, S, or I'
    user_clouds_tries = 3
    user_clouds_default = 'I'
    user_clouds_input = valid_input(user_clouds_q, user_clouds_allowed, user_clouds_err_msg, user_clouds_tries, user_clouds_default)
    ideal_clouds = user_clouds_input[1]
else:
    ideal_temp = '70'
    ideal_humid = 'I'
    ideal_wind = 'I'
    ideal_clouds = 'I'


Default vacation spot search criteria are: 70F, 40% humidity, and any level of windiness or cloudiness. Would you like to customize your search (y/n) n


In [106]:
#beaufort_scale_wind = [0, 7, 12, 18, 24, 38, 54, 72, 125]
#wind_bins = ['Light', 'Gentle', 'Moderate', 'Fresh', 'Strong', 'Gale', 'Whole Gale', 'Hurricane']
wind_tolerance_dict = {'G': 12, 'F':24, 'I':125}
wind_desc_dict = {'G': "gentle", 'F':'fresh', 'I':'any'}

#Cloudiness scale in normal terms
#cloud_scale = [-1, 10, 50, 90, 100]
#cloud_bins = ['Clear', 'Scattered', 'Broken', 'Overcast']
cloud_tolerance_dict = {'C': 10, 'S':50, 'I':100}
cloud_desc_dict = {'C': 'clear', 'S':'scattered', 'I':'any'}

#user_inputs_df = pd.DataFrame('Temperature (F)': ideal_temp)
user_inputs_df = pd.DataFrame({'Temperature':[ideal_temp], 'Humidity':[ideal_humid], 'Wind':wind_desc_dict[ideal_wind], 'Clouds':cloud_desc_dict[ideal_clouds]})

#Assume 10F degress +/- is and acceptable tolerance level to ideal
temp_tolerance = 10 
if ideal_humid != 'I':
    ideal_humid = int(ideal_humid)
    humid_tolerance = 10
    upper_humid = ideal_humid + humid_tolerance
    lower_humid = ideal_humid - humid_tolerance  
else:
    upper_humid = 100
    lower_humid = 0
    humid_tolerance = 0

upper_temp = int(ideal_temp) + temp_tolerance
lower_temp = int(ideal_temp) - temp_tolerance 

cities_choice_df = cities_weather_df.loc[cities_weather_df['Temperature (F)']<upper_temp]
cities_choice_df = cities_choice_df.loc[cities_choice_df['Temperature (F)']>lower_temp]
cities_choice_df = cities_choice_df.loc[cities_choice_df['Humidity']<upper_humid]
cities_choice_df = cities_choice_df.loc[cities_choice_df['Humidity']>lower_humid]
cities_choice_df = cities_choice_df.loc[cities_choice_df['Wind Speed']<wind_tolerance_dict[ideal_wind]]
cities_choice_df = cities_choice_df.loc[cities_choice_df['Cloudiness']<=cloud_tolerance_dict[ideal_clouds]]

orig_num_cities = cities_choice_df['City ID'].count()

num_cities = cities_choice_df['City ID'].count()

while num_cities>10 and temp_tolerance>1 and humid_tolerance>1:
    temp_tolerance = temp_tolerance*0.9
    humid_tolerance = humid_tolerance*0.9
    upper_temp = int(ideal_temp) + temp_tolerance
    lower_temp = int(ideal_temp) - temp_tolerance  
    upper_humid = int(ideal_humid) + humid_tolerance
    lower_humid = int(ideal_humid) - humid_tolerance  
    cities_choice_df = cities_choice_df.loc[cities_choice_df['Temperature (F)']<upper_temp]
    cities_choice_df = cities_choice_df.loc[cities_choice_df['Temperature (F)']>lower_temp]
    cities_choice_df = cities_choice_df.loc[cities_choice_df['Humidity']<upper_humid]
    cities_choice_df = cities_choice_df.loc[cities_choice_df['Humidity']>lower_humid]
    num_cities = cities_choice_df['City ID'].count()

if num_cities>10:
    cities_choice_df = cities_choice_df.sample(10)

print(f'We have selected {num_cities} cities that meet your criteria.')

print(orig_num_cities)
user_inputs_df.index = ['Vacation City Criteria']
user_inputs_df


We have selected 9 cities that meet your criteria.
14


,Temperature,Humidity,Wind,Clouds
Vacation City Criteria,70,40,any,any


In [101]:
print(f'Please see below a summary of the {num_cities} cities selected.')
cities_choice_df.head(20).style.hide_index()


Please see below a summary of the 9 cities selected.


City ID,City,Country,Act Lat,Act Lon,Temperature (F),Humidity,Cloudiness,Wind Speed
3896218,Castro,CL,-42.470000,-73.770000,73.400000,35,0,4.700000
4500920,Brigantine,US,39.410000,-74.360000,71.170000,43,20,11.410000
187585,Marsabit,KE,2.330000,37.990000,73.560000,46,58,13.910000
3995236,Mulege,MX,26.880000,-111.980000,73.000000,45,100,3.910000
3985710,Cabo san lucas,MX,22.890000,-109.910000,77.220000,44,90,4.700000
3899695,Ancud,CL,-41.870000,-73.820000,73.400000,35,0,4.700000
1263532,Margherita,IN,27.280000,95.680000,75.000000,34,53,2.420000
3353383,Rundu,NA,-17.930000,19.770000,73.240000,40,49,3.200000
3868707,Valdivia,CL,-39.810000,-73.250000,73.400000,43,0,4.700000


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = cities_choice_df
hotel_names =[]
hotel_addresses = []
hotel_lat = []
hotel_lng = []

city_lat_long = zip(hotel_df['Act Lat'], hotel_df['Act Lon'])

biz_type = 'hotel'
biz_radius = 5000

for lat_lng in city_lat_long:
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'  
    loc_url = 'location='+str(lat_lng[0])+','+str(lat_lng[1])
    radius_url = '&radius='+str(biz_radius)
    type_url = '&type='+biz_type
    api_key_url = '&key='+g_key
    response_url = base_url + loc_url + radius_url + type_url + api_key_url
    response = requests.get(response_url).json()
    #print(json.dumps(response, indent=4, sort_keys=False))
    first_hotel_name = response['results'][1]['name']
    hotel_names.append(first_hotel_name)
    try:
        first_hotel_address = response['results'][1]['vicinity']
    except: 
        first_hotel_address = 'No Address Provided'
    hotel_addresses.append(first_hotel_address)
    first_hotel_lat = response['results'][1]['geometry']['location']['lat']
    first_hotel_lng = response['results'][1]['geometry']['location']['lng']
    hotel_lat.append(first_hotel_lat)
    hotel_lng.append(first_hotel_lng)


In [8]:
hotel_df['Hotel Name']=hotel_names
hotel_df['Hotel Address']= hotel_addresses
hotel_df['Hotel Lat']=hotel_lat
hotel_df['Hotel Lon']=hotel_lng
hotel_df

,City ID,City,Country,Act Lat,Act Lon,Temperature (F),Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Address,Hotel Lat,Hotel Lon
94,1807119,Huangzhai,CN,38.08,112.66,44.60,45,93,13.42,Yangqu Agricultural Bureau,"首邑西路, 108国道, Yangqu County, Taiyuan",38.051999,112.665676
110,1253628,Lata,SB,30.78,78.62,46.24,24,47,6.85,SBI ATM,"Uttarkashi - Gangotri Road, Bhatwari",30.817297,78.618422
206,609919,Kuryk,KZ,43.20,51.65,43.52,62,0,9.78,Kazkommertsbank,Kuryk,43.178659,51.670464
240,1848976,Wajima,JP,37.40,136.90,44.89,69,85,11.63,Wajiima Onsen Yashio,"Sodegahama-1 Fugeshimachi, Wajima",37.401661,136.896323
245,546105,Nikolskoye,RU,59.70,30.79,43.36,33,40,2.24,Nikolskaya College № 1,"Ulitsa Shkol'naya, 11, Тосненский р-н, г. Нико...",59.703497,30.792114
258,2181625,Te anau,NZ,-45.42,167.72,43.93,73,100,3.42,Kingsgate Hotel Te Anau,"20 Lakefront Drive, Te Anau",-45.422450,167.717546
317,1812256,Dongsheng,CN,39.82,109.98,43.47,29,51,8.41,Dongsheng Sports Center Experience Center,"Minzu West Street, Dongsheng District, Ordos City",39.832720,109.954353
364,553766,Kamenka,RU,51.32,42.77,45.88,82,77,9.44,Kamenskaya Oosh,Kamenka,51.320956,42.762272
387,602150,Umea,SE,63.83,20.26,44.60,52,99,10.29,Hotel Avenue | ProfilHotels,"Rådhusesplanaden 14, Umeå",63.828389,20.265893


In [9]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lon"]]

# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…